## Summing attention distributions from the last four layers then classifying via a Linear Network

We extract the attention distribution directly from a model that has been pretrained on bias detection!

We then determine the predicted biased word, and use only the attention distribution associated with that extracted word for further classification. 

We sum together the last four attention distributions of the input. The classification model is a simple super shallow neural network.

In [4]:
import sys; sys.path.append("../../../../..")
import torch 
from src.experiment import AttentionExperiment, ClassificationExperiment
from src.dataset import ExperimentDataset
from src.params import Params
from src.utils.attention_utils import reduce_attention_dist, return_idx_attention_dist
from src.utils.classification_utils import run_bootstrapping
from src.utils.shared_utils import get_bias_predictions

In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
params = Params.read_params("linear-params.json")
print("model = {}".format(params.final_task['model']))
print("layers = {}".format(params.intermediary_task["attention"]["layers"]))
print("reducer = {}".format(params.intermediary_task["attention"]["reducer"]))

model = shallow_nn
layers = [8, 9, 10, 11]
reducer = sum


In [7]:
# Loading in the dataset that we are using in this experiments 
# typically this dataset is the small set of ground-truth labels
dataset = ExperimentDataset.init_dataset(params.dataset)

03/21/2020 17:43:10 - INFO - pytorch_pretrained_bert.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt not found in cache, downloading to /tmp/tmpcx9k74nj
100%|██████████| 231508/231508 [00:00<00:00, 1184282.98B/s]
03/21/2020 17:43:10 - INFO - pytorch_pretrained_bert.file_utils -   copying /tmp/tmpcx9k74nj to cache at ./cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
03/21/2020 17:43:10 - INFO - pytorch_pretrained_bert.file_utils -   creating metadata file for ./cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
03/21/2020 17:43:10 - INFO - pytorch_pretrained_bert.file_utils -   removing temp file /tmp/tmpcx9k74nj
03/21/2020 17:43:10 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-unca

### Attention Experiment: 
* Is a class that wraps useful methods to extract attention distributions from a given BERT-based model 
* The user has to provide in two config files: One to specify parameters for how the attention scores should be extracted and combined, and other to specify the intermediary model from which the attention scores should be extracted from
* The user needs to instantiate the attention experiment with a function that tells the model how to run 
 inference on the given model. The function header is specified below: 
 
 ``` def initialize_attention_experiment(cls, intermediary_task_params, dataset_params, verbose=False) ```
 


In [8]:
attention_dataloader = dataset.return_dataloader() 
attention_experiment = AttentionExperiment.initialize_attention_experiment(params.intermediary_task, params.dataset, verbose=True)

03/21/2020 17:43:11 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
03/21/2020 17:43:11 - INFO - pytorch_pretrained_bert.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz not found in cache, downloading to /tmp/tmpfj_6na37
100%|██████████| 407873900/407873900 [00:24<00:00, 16353095.80B/s]
03/21/2020 17:43:36 - INFO - pytorch_pretrained_bert.file_utils -   copying /tmp/tmpfj_6na37 to cache at ./cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
03/21/2020 17:43:40 - INFO - pytorch_pretrained_bert.file_utils -   creating metadata file for ./cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0

Instantiated joint model with pretrained weights.
Succesfully loaded in attention experiment!


```extract_attention_scores()``` works out of the box because the attention experiment has the config file saved, and knows what BERT model to use/load in, which layers to extract the attention scores from, and what the inference function is that should be used on this particular BERT model.

Attention_scores is then a list of dictionaries. The keys in this dictionary are the specific layers of a BERT model and the values are the corresponding attention distributions extracted from that particular layer.

In [ ]:
attention_scores = attention_experiment.extract_attention_scores(attention_dataloader)

Getting the predictions from the BERT model trained to detect bias, and using those to index into the attention scores

In [ ]:
bias_predictions = get_bias_predictions(dataset, params.intermediary_task, params.dataset, batch_size=2)

03/21/2020 17:44:14 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
03/21/2020 17:44:14 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at ./cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
03/21/2020 17:44:14 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file ./cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpdm0c7l8w
03/21/2020 17:44:19 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_d

In [ ]:
bias_indices = torch.argmax(bias_predictions == 1, dim=1).tolist()

In [ ]:
attention_scores_indexed = return_idx_attention_dist(attention_scores, bias_indices)
reduced_attention = reduce_attention_dist(attention_scores_indexed, params.intermediary_task["attention"]["reducer"])
stacked_reduced_attention = torch.stack(reduced_attention).squeeze()
dataset.add_data(stacked_reduced_attention, "attention_dist")
dataset.shuffle_data()
print("reduced_attention[0].shape = {}".format(reduced_attention[0].shape))
print("stacked_reduced_attention.shape = {}".format(stacked_reduced_attention.shape))
dataset

### This is where the classification experiment starts

We create a classification experiment that contains useful methods for classifying bias based on the attention distributions. 

In [ ]:
params = Params.read_params("linear-params.json")


In [ ]:
classification_experiment = ClassificationExperiment.init_cls_experiment(params.final_task)

In [ ]:
stats = run_bootstrapping(classification_experiment, dataset, params.final_task, num_bootstrap_iters=5, input_key="attention_dist", label_key="bias_label")

120 Epochs - 15 hidden

In [ ]:
stats

In [ ]:
classification_experiment.save_model_weights("linear-attention.weights")